## Experiencia III: Minería de Texto
### Taller de Minería de Datos Avanzados
### Profesores:
### Dr. Max Chacón Pacheco
### Dr. Felipe Bello Robles
### Experiencia realizada por Miguel Salinas González




Importación de librerías a utilizar.

RTextTools se utiliza para acceder a las herramientas de minería de texto.

lime se utiliza para ahorrar la lectura de los datos, ya que en lime se cuenta con los datos de Sentences Corpus de UCI (tanto de entrenamiento como de validación).

In [24]:
#install.packages("RTextTools")
#install.packages("lime")
library(RTextTools)
library(lime)

also installing the dependencies 'colorspace', 'farver', 'labeling', 'munsell', 'RColorBrewer', 'viridisLite', 'gtable', 'isoband', 'scales', 'ggplot2', 'assertthat', 'gower'





The downloaded binary packages are in
	/var/folders/t6/_6720_rx5cvf7sc1r110_6f80000gn/T//RtmpvVVDfE/downloaded_packages


### Preprocesamiento de los datos.

Primeramente se accede a los datos de Sentences Corpus disponibles en la librería lime. Dentro de dicha librería también se cuenta con una lista de palabras llamadas stopwords, esta lista considera palabras las cuales son seguras de eliminar en el proceso de minería. El proceso llevado a cabo con esto es eliminar dichas palabras de cada bloque de texto en el set de datos.

En segundo lugar, se hace reemplazo de las clases por valores numéricos. En este proceso se identifico casos donde la lectura de los datos había estado mal realizada, resultado con etiquetas del tipo "MISC --in", este tipo de etiquetas al ser identificadas en el proceso, son corregidas manualmente. Terminando en la siguiente glosa:

CONT = 4

MISC = 6

AIMX = 1

OWNX = 13

BASE = 3

Finalmente se muestra la distribución de cada una de las clases en las instancias.

In [205]:
data(test_sentences)
stopwords <- stop_words_sentences
test <- test_sentences
train <- train_sentences
for (palabra in stopwords) {
  test$text <- gsub(paste0("\\b", palabra, "\\b"), "", test$text)
  train$text <- gsub(paste0("\\b", palabra, "\\b"), "", train$text)
}

df_final <- rbind(train, test)
df_final$text <- as.factor(df_final$text)
df_final$class.text <- as.factor(df_final$class.text)

# CORRECCIÓN DE CLASE 
# En algunos casos la etiqueta era agregada con la primera palabra del texto
# Ejemplo "MISC --in"
# La corrección se hace manual con la función which
df_final[1194,]$class.text <- "MISC"
df_final[3061,]$class.text <- "MISC"
df_final[1187,]$class.text <- "CONT"
df_final[2701,]$class.text <- "CONT"
df_final[1180,]$class.text <- "MISC"
df_final[1189,]$class.text <- "MISC"
df_final[1193,]$class.text <- "MISC"
df_final[2828,]$class.text <- "MISC"
df_final[1178,]$class.text <- "AIMX"
df_final[1186,]$class.text <- "AIMX"
df_final[1191,]$class.text <- "AIMX"
df_final[1200,]$class.text <- "AIMX"
df_final[1184,]$class.text <- "MISC"
df_final[1199,]$class.text <- "MISC"
df_final[1185,]$class.text <- "MISC"
df_final[1179,]$class.text <- "OWNX"
df_final[1192,]$class.text <- "OWNX"
df_final[1198,]$class.text <- "MISC"
df_final[1183,]$class.text <- "MISC"
df_final[1197,]$class.text <- "MISC"
df_final[1177,]$class.text <- "MISC"
df_final[1181,]$class.text <- "MISC"
df_final[1182,]$class.text <- "MISC"
df_final[1190,]$class.text <- "MISC"
df_final[1195,]$class.text <- "MISC"
df_final[1196,]$class.text <- "MISC"
df_final[1188,]$class.text <- "OWNX"
df_final[1201,]$class.text <- "OWNX"
df_final$class.text <- as.numeric(df_final$class.text)
# CONT = 4
# MISC = 6
# AIMX = 1
# OWNX = 13
# BASE = 3
#which(df_final$class.text == 15)

recuento_clases <- table(df_final$class.text)
df_recuento_clases <- as.data.frame(recuento_clases)
df_recuento_clases

Var1,Freq
<fct>,<int>
1,194
3,61
4,170
6,1825
13,867


### Random Forest
Primeramente se aplica el algoritmo de Random Forest en el proceso de minería de texto, para esto se hacen diversas pruebas para ajustar correctamente los parámetros. Es en este último proceso donde se indentifica que alrededor de los 300 árboles generados se obtienen los mismos resultados para las clases más frecuentas (MISC y OWNX), variando levemente en el resto. Basándose en el principio de parsimonia este es el modelo escogido para la técnica de RandomForest.

In [215]:
doc_matrix <- create_matrix(df_final$text, language="english", removeNumbers=FALSE,stemWords=FALSE, removeSparseTerms=.998) #Usa package tm para preprocesamiento.

container <- create_container(doc_matrix, df_final$class.text, trainSize=1:2517, testSize=2518:3117, virgin=FALSE)#divide conjunto en entrenamiento y validacion
RFmodel <- train_model(container, algorithm="RF", ntree=300)
RFresults <- classify_model(container, RFmodel)

# VIEW THE RESULTS BY CREATING ANALYTICS
RFanalytics <- create_analytics(container, RFresults)
RFanalytics@algorithm_summary #: SUMMARY OF PRECISION, RECALL, F-SCORES, AND ACCURACY SORTED BY TOPIC CODE FOR EACH ALGORITHM
RFanalytics@label_summary     #: SUMMARY OF LABEL (e.g. TOPIC) ACCURACY
# Se comenta para  reducir los outputs, pero básicamente muestra como fue clasificada muestra
#RFanalytics@document_summary  #: RAW SUMMARY OF ALL DATA AND SCORING
RFanalytics@ensemble_summary  #: SUMMARY OF ENSEMBLE PRECISION/COVERAGE. USES THE n VARIABLE PASSED INTO create_analytics()

Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom functions are ignored"
Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom tokenizer is ignored"


,FORESTS_PRECISION,FORESTS_RECALL,FORESTS_FSCORE
,<dbl>,<dbl>,<dbl>
1,0.90,0.66,0.76
3,0.60,0.46,0.52
4,0.50,0.44,0.47
6,0.90,0.94,0.92
13,0.81,0.82,0.81


,NUM_MANUALLY_CODED,NUM_CONSENSUS_CODED,NUM_PROBABILITY_CODED,PCT_CONSENSUS_CODED,PCT_PROBABILITY_CODED,PCT_CORRECTLY_CODED_CONSENSUS,PCT_CORRECTLY_CODED_PROBABILITY
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,41,30,30,73.17073,73.17073,65.85366,65.85366
3,13,10,10,76.92308,76.92308,46.15385,46.15385
4,25,22,22,88.00000,88.00000,44.00000,44.00000
6,366,382,382,104.37158,104.37158,94.26230,94.26230
13,155,156,156,100.64516,100.64516,81.93548,81.93548


,n-ENSEMBLE COVERAGE,n-ENSEMBLE RECALL
n >= 1,1,0.86


### SLDA
En esta fase de nuestra investigación de minería de texto, implementamos el algoritmo Supervised Latent Dirichlet Allocation (SLDA) para descubrir temas subyacentes y clasificar los documentos de forma supervisada. A través de una serie de iteraciones y ajustes finos, determinamos que la configuración de parámetros que excluye números y no aplica stemming, junto con la eliminación de términos que aparecen en un porcentaje muy bajo de documentos, conduce a un modelo que equilibra adecuadamente la generalización y el sobreajuste. Esta configuración, que también contempla un balance entre la cantidad de datos de entrenamiento y prueba, refleja el principio de parsimonia, permitiéndonos seleccionar un modelo SLDA eficiente y efectivo. 

In [209]:
doc_matrix <- create_matrix(df_final$text, language="english", removeNumbers=FALSE,stemWords=FALSE, removeSparseTerms=.998) #Usa package tm para preprocesamiento.


container <- create_container(doc_matrix, df_final$class.text, trainSize=1:2517, testSize=2518:3117, virgin=FALSE)
#container <- create_container(doc_matrix, df_final$class.text, trainSize=1:200, testSize=201:300, virgin=FALSE) #divide conjunto en entrenamiento y validacion
RFmodel <- train_model(container, algorithm="SLDA")
RFresults <- classify_model(container, RFmodel)


# VIEW THE RESULTS BY CREATING ANALYTICS
RFanalytics <- create_analytics(container, RFresults)
RFanalytics@algorithm_summary #: SUMMARY OF PRECISION, RECALL, F-SCORES, AND ACCURACY SORTED BY TOPIC CODE FOR EACH ALGORITHM
RFanalytics@label_summary     #: SUMMARY OF LABEL (e.g. TOPIC) ACCURACY
# Se comenta para  reducir los outputs, pero básicamente muestra como fue clasificada muestra
#RFanalytics@document_summary  #: RAW SUMMARY OF ALL DATA AND SCORING
RFanalytics@ensemble_summary  #: SUMMARY OF ENSEMBLE PRECISION/COVERAGE. USES THE n VARIABLE PASSED INTO create_analytics()

Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom functions are ignored"
Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom tokenizer is ignored"


,SLDA_PRECISION,SLDA_RECALL,SLDA_FSCORE
,<dbl>,<dbl>,<dbl>
1,0.90,0.63,0.74
3,0.62,0.62,0.62
4,0.50,0.36,0.42
6,0.85,0.92,0.88
13,0.73,0.68,0.70


,NUM_MANUALLY_CODED,NUM_CONSENSUS_CODED,NUM_PROBABILITY_CODED,PCT_CONSENSUS_CODED,PCT_PROBABILITY_CODED,PCT_CORRECTLY_CODED_CONSENSUS,PCT_CORRECTLY_CODED_PROBABILITY
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,41,29,29,70.73171,70.73171,63.41463,63.41463
3,13,13,13,100.00000,100.00000,61.53846,61.53846
4,25,18,18,72.00000,72.00000,36.00000,36.00000
6,366,397,397,108.46995,108.46995,91.80328,91.80328
13,155,143,143,92.25806,92.25806,67.74194,67.74194


,SLDA_LABEL,SLDA_PROB,MANUAL_CODE,CONSENSUS_CODE,CONSENSUS_AGREE,CONSENSUS_INCORRECT,PROBABILITY_CODE,PROBABILITY_INCORRECT
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,13,0.9971899,13,13,1,0,13,0
2,6,0.9107058,6,6,1,0,6,0
3,6,0.9748083,6,6,1,0,6,0
4,6,0.9781834,13,6,1,1,6,1
5,1,0.9968351,13,1,1,1,1,1
6,6,0.9486638,6,6,1,0,6,0
7,6,0.9984901,6,6,1,0,6,0
8,6,0.9799097,6,6,1,0,6,0
9,13,0.9150264,6,13,1,1,13,1


,n-ENSEMBLE COVERAGE,n-ENSEMBLE RECALL
n >= 1,1,0.81


### GLMNET
Se hace uso del algoritmo 'GLMNET', un modelo de regresión de red elástica, con un número de iteraciones suficientemente grande para garantizar la convergencia. Una vez entrenado el modelo y clasificados los datos, procedemos a evaluar el rendimiento a través de un conjunto de analíticas que incluyen resúmenes detallados de precisión, recall, puntuaciones F1 y exactitud por cada etiqueta temática.

In [214]:
doc_matrix <- create_matrix(df_final$text, language="english", removeNumbers=FALSE,stemWords=FALSE, removeSparseTerms=.998) #Usa package tm para preprocesamiento.


container <- create_container(doc_matrix, df_final$class.text, trainSize=1:2517, testSize=2518:3117, virgin=FALSE)
RFmodel <- train_model(container, algorithm="GLMNET",maxitglm=10^9)
RFresults <- classify_model(container, RFmodel)


# VIEW THE RESULTS BY CREATING ANALYTICS
RFanalytics <- create_analytics(container, RFresults)
RFanalytics@algorithm_summary #: SUMMARY OF PRECISION, RECALL, F-SCORES, AND ACCURACY SORTED BY TOPIC CODE FOR EACH ALGORITHM
RFanalytics@label_summary     #: SUMMARY OF LABEL (e.g. TOPIC) ACCURACY
#RFanalytics@document_summary  #: RAW SUMMARY OF ALL DATA AND SCORING
RFanalytics@ensemble_summary  #: SUMMARY OF ENSEMBLE PRECISION/COVERAGE. USES THE n VARIABLE PASSED INTO create_analytics()

Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom functions are ignored"
Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom tokenizer is ignored"


,GLMNET_PRECISION,GLMNET_RECALL,GLMNET_FSCORE
,<dbl>,<dbl>,<dbl>
1,0.96,0.56,0.71
3,0.33,0.08,0.13
4,0.50,0.12,0.19
6,0.83,0.98,0.90
13,0.79,0.69,0.74


,NUM_MANUALLY_CODED,NUM_CONSENSUS_CODED,NUM_PROBABILITY_CODED,PCT_CONSENSUS_CODED,PCT_PROBABILITY_CODED,PCT_CORRECTLY_CODED_CONSENSUS,PCT_CORRECTLY_CODED_PROBABILITY
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,41,24,24,58.53659,58.53659,56.097561,56.097561
3,13,3,3,23.07692,23.07692,7.692308,7.692308
4,25,6,6,24.00000,24.00000,12.000000,12.000000
6,366,432,432,118.03279,118.03279,98.087432,98.087432
13,155,135,135,87.09677,87.09677,69.032258,69.032258


GLMNET_LABEL,GLMNET_PROB,MANUAL_CODE,CONSENSUS_CODE,CONSENSUS_AGREE,CONSENSUS_INCORRECT,PROBABILITY_CODE,PROBABILITY_INCORRECT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
13,0.9282195,13,13,1,0,13,0
6,0.7933421,6,6,1,0,6,0
6,0.8282542,6,6,1,0,6,0
6,0.6386162,13,6,1,1,6,1
6,0.4096544,13,6,1,1,6,1
6,0.7539999,6,6,1,0,6,0
6,0.7101591,6,6,1,0,6,0
6,0.8971768,6,6,1,0,6,0
6,0.4027247,6,6,1,0,6,0


,n-ENSEMBLE COVERAGE,n-ENSEMBLE RECALL
n >= 1,1,0.82
